In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, f1_score, recall_score, precision_score

In [65]:
df = pd.read_csv('./second.csv', index_col=0)
ddf = pd.read_csv('./second.csv', index_col=0)

In [3]:
ddf.head()

,ibk_seq,id_seq,base_ym,default_tf,ceo_name_cnt,bsn_opr_month,ksic_code,corporate_type,count,address,...,utility_cnt,utility_reg_num_tf,utility_reg_num_opr_month,design_cnt,design_app_date_tf,cprgt_cnt,total,avg,pulse_grade,article_cnt
1,1,769252,2018-11-30,0,1,679,M71310,주식회사,0,서울특별시 강남구,...,0,0,0,0,0,0,5,2,5,0
2,2,769252,2018-12-31,0,1,679,M71310,주식회사,0,서울특별시 강남구,...,0,0,0,0,0,0,5,2,5,0
3,3,769252,2019-01-31,0,1,679,M71310,주식회사,0,서울특별시 강남구,...,0,0,0,0,0,0,5,2,5,0
4,4,769252,2019-02-28,0,1,679,M71310,주식회사,0,서울특별시 강남구,...,0,0,0,0,0,0,5,2,5,0
5,5,769252,2019-03-31,0,1,679,M71310,주식회사,0,서울특별시 강남구,...,0,0,0,0,0,0,5,2,5,0


## 1. 마지막거 남기기

In [4]:
df = df.drop_duplicates(['id_seq'], keep='last')

In [5]:
df = df.drop('base_ym', axis=1)
df = df.drop(['ibk_seq'], axis=1)

# df = df.drop('exporter_tf', axis=1)
# df = df.drop('export_potent_tf', axis=1)
# df = df.drop('ysbg_tf', axis=1)
# df = df.drop('majority_retired_tf', axis=1)
# df = df.drop('utility_cnt', axis=1)
# df = df.drop('design_app_date_tf', axis=1)

# df = df.drop('smallgiant_brand_tf', axis=1)
# df = df.drop('cntrct_cncls_date_tf', axis=1)
# df = df.drop('utility_reg_num_tf', axis=1)

df['bsn_opr_month'] = np.log1p(df['bsn_opr_month'])
df['pblntf_pclnd'] = np.log1p(df['pblntf_pclnd'])
df['bsn_history_cnt'] = np.log1p(df['bsn_history_cnt'])
df=df.fillna(0)

df = pd.get_dummies(df)

C:\Users\pc\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [66]:
ddf = ddf.drop('base_ym', axis=1)
ddf = ddf.drop(['ibk_seq'], axis=1)

# ddf = ddf.drop('exporter_tf', axis=1)
# ddf = ddf.drop('export_potent_tf', axis=1)
# ddf = ddf.drop('ysbg_tf', axis=1)
# ddf = ddf.drop('majority_retired_tf', axis=1)
# ddf = ddf.drop('utility_cnt', axis=1)
# ddf = ddf.drop('design_app_date_tf', axis=1)

ddf = ddf.drop('smallgiant_brand_tf', axis=1)
ddf = ddf.drop('cntrct_cncls_date_tf', axis=1)
ddf = ddf.drop('utility_reg_num_tf', axis=1)

ddf['bsn_opr_month'] = np.log1p(ddf['bsn_opr_month'])
ddf['pblntf_pclnd'] = np.log1p(ddf['pblntf_pclnd'])
ddf['bsn_history_cnt'] = np.log1p(ddf['bsn_history_cnt'])
ddf=ddf.fillna(0)

ddf = pd.get_dummies(ddf)

C:\Users\pc\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


## 2. 그중에 0, 1 별로 모으기

In [7]:
norma = df[df['default_tf']==0]
targe = df[df['default_tf']==1]
print(int(len(norma)*0.75), int(len(targe)*0.75))

31794 1074


## 3. 각각 모으기

In [8]:
n_train = norma[:31794]
n_test = norma[31794:]

b_train = targe[:1074]
b_test = targe[1074:]

In [9]:
train = pd.concat([n_train,b_train])
test = pd.concat([n_test,b_test])

## 4. id 리스트만들기

In [10]:
train_id = train['id_seq']
test_id = test['id_seq']

In [11]:
train_id.reset_index(drop=True, inplace=True)
test_id.reset_index(drop=True, inplace=True)

In [12]:
train_id

0        769252
1         65967
2        891725
3        321236
4        294699
          ...  
32863    137071
32864    592259
32865    479059
32866    363024
32867    980329
Name: id_seq, Length: 32868, dtype: int64

## 5. df를 나누기

In [13]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()

for i in range(len(train_id)):
    ddff = ddf[ddf['id_seq']==train_id[i]]
    train_df = pd.concat([train_df,ddff])

In [14]:
for i in range(len(test_id)):
    ddff = ddf[ddf['id_seq']==test_id[i]]
    test_df = pd.concat([test_df,ddff])

In [15]:
len(train_id), len(train_df)

(32868, 180116)

In [16]:
len(test_id), len(test_df)

(10956, 58554)

In [17]:
len(ddf)

238670

In [18]:
train_df.to_csv('./train_original.csv')
test_df.to_csv('./test_original.csv')

# 시작

In [68]:
train_df = pd.read_csv('./train_original.csv', index_col=0)
test_df = pd.read_csv('./test_original.csv', index_col=0)

In [69]:
train_df = train_df.drop('id_seq', axis=1)
test_df = test_df.drop('id_seq', axis=1)

train_df = train_df.drop('smallgiant_brand_tf', axis=1)
train_df = train_df.drop('cntrct_cncls_date_tf', axis=1)
train_df = train_df.drop('utility_reg_num_tf', axis=1)

test_df = test_df.drop('smallgiant_brand_tf', axis=1)
test_df = test_df.drop('cntrct_cncls_date_tf', axis=1)
test_df = test_df.drop('utility_reg_num_tf', axis=1)

# train_df = train_df.drop('exporter_tf', axis=1)
# train_df = train_df.drop('export_potent_tf', axis=1)
# train_df = train_df.drop('ysbg_tf', axis=1)
# train_df = train_df.drop('majority_retired_tf', axis=1)
# train_df = train_df.drop('utility_cnt', axis=1)
# train_df = train_df.drop('design_app_date_tf', axis=1)

# test_df = test_df.drop('exporter_tf', axis=1)
# test_df = test_df.drop('export_potent_tf', axis=1)
# test_df = test_df.drop('ysbg_tf', axis=1)
# test_df = test_df.drop('majority_retired_tf', axis=1)
# test_df = test_df.drop('utility_cnt', axis=1)
# test_df = test_df.drop('design_app_date_tf', axis=1)

In [70]:
y_train = train_df['default_tf']
x_train = train_df.drop('default_tf', axis=1)

y_test = test_df['default_tf']
x_test = test_df.drop('default_tf', axis=1)

# catboost

In [22]:
!pip install catboost

# 오버샘플링

In [50]:
from sklearn.model_selection import train_test_split

In [24]:
# print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
# print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

In [25]:
# pip install imbalanced-learn

In [26]:
# pip install scikit-learn==1.1

In [27]:
# from imblearn.over_sampling import SMOTE
# sm = SMOTE(random_state=2)
# X_res, y_res = sm.fit_resample(x_train, y_train.ravel())

# print('After OverSampling, the shape of X: {}'.format(X_res.shape))
# print('After OverSampling, the shape of y: {} \n'.format(y_res.shape))

# print("After OverSampling, counts of label '1': {}".format(sum(y_res==1)))
# print("After OverSampling, counts of label '0': {}".format(sum(y_res==0)))

In [28]:
# x_train, x_valid, y_train, y_valid = train_test_split(X_res, y_res, shuffle=True, stratify=y_res, random_state=42)

# catboost

In [71]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier()

In [72]:
cat.fit(x_train, y_train)

Learning rate set to 0.094636
0:	learn: 0.5352782	total: 57.1ms	remaining: 57s
1:	learn: 0.4137372	total: 101ms	remaining: 50.3s
2:	learn: 0.3306233	total: 146ms	remaining: 48.5s
3:	learn: 0.2713231	total: 189ms	remaining: 47.1s
4:	learn: 0.2281991	total: 232ms	remaining: 46.2s
5:	learn: 0.1967704	total: 279ms	remaining: 46.2s
6:	learn: 0.1718233	total: 326ms	remaining: 46.3s
7:	learn: 0.1517735	total: 384ms	remaining: 47.6s
8:	learn: 0.1357679	total: 440ms	remaining: 48.5s
9:	learn: 0.1253279	total: 486ms	remaining: 48.1s
10:	learn: 0.1163795	total: 542ms	remaining: 48.7s
11:	learn: 0.1107435	total: 588ms	remaining: 48.4s
12:	learn: 0.1065625	total: 639ms	remaining: 48.5s
13:	learn: 0.1028658	total: 693ms	remaining: 48.8s
14:	learn: 0.1000093	total: 740ms	remaining: 48.6s
15:	learn: 0.0972038	total: 799ms	remaining: 49.1s
16:	learn: 0.0953585	total: 848ms	remaining: 49.1s
17:	learn: 0.0936197	total: 897ms	remaining: 48.9s
18:	learn: 0.0925073	total: 946ms	remaining: 48.8s
19:	learn: 0

164:	learn: 0.0742603	total: 8.53s	remaining: 43.2s
165:	learn: 0.0741844	total: 8.58s	remaining: 43.1s
166:	learn: 0.0741493	total: 8.63s	remaining: 43s
167:	learn: 0.0740791	total: 8.68s	remaining: 43s
168:	learn: 0.0740144	total: 8.73s	remaining: 42.9s
169:	learn: 0.0739668	total: 8.78s	remaining: 42.9s
170:	learn: 0.0738809	total: 8.84s	remaining: 42.9s
171:	learn: 0.0737984	total: 8.9s	remaining: 42.9s
172:	learn: 0.0737374	total: 8.96s	remaining: 42.8s
173:	learn: 0.0736694	total: 9.01s	remaining: 42.8s
174:	learn: 0.0736282	total: 9.07s	remaining: 42.8s
175:	learn: 0.0735575	total: 9.13s	remaining: 42.7s
176:	learn: 0.0735289	total: 9.18s	remaining: 42.7s
177:	learn: 0.0734734	total: 9.23s	remaining: 42.6s
178:	learn: 0.0733912	total: 9.28s	remaining: 42.6s
179:	learn: 0.0733482	total: 9.33s	remaining: 42.5s
180:	learn: 0.0732643	total: 9.39s	remaining: 42.5s
181:	learn: 0.0731864	total: 9.45s	remaining: 42.5s
182:	learn: 0.0731400	total: 9.5s	remaining: 42.4s
183:	learn: 0.0730

323:	learn: 0.0662966	total: 16.2s	remaining: 33.8s
324:	learn: 0.0662691	total: 16.2s	remaining: 33.7s
325:	learn: 0.0662040	total: 16.3s	remaining: 33.7s
326:	learn: 0.0661515	total: 16.4s	remaining: 33.6s
327:	learn: 0.0660985	total: 16.4s	remaining: 33.6s
328:	learn: 0.0660804	total: 16.4s	remaining: 33.5s
329:	learn: 0.0660298	total: 16.5s	remaining: 33.5s
330:	learn: 0.0659913	total: 16.5s	remaining: 33.4s
331:	learn: 0.0659334	total: 16.6s	remaining: 33.3s
332:	learn: 0.0658949	total: 16.6s	remaining: 33.3s
333:	learn: 0.0658611	total: 16.7s	remaining: 33.2s
334:	learn: 0.0658288	total: 16.7s	remaining: 33.2s
335:	learn: 0.0657789	total: 16.7s	remaining: 33.1s
336:	learn: 0.0657433	total: 16.8s	remaining: 33s
337:	learn: 0.0657210	total: 16.8s	remaining: 33s
338:	learn: 0.0656745	total: 16.9s	remaining: 32.9s
339:	learn: 0.0656532	total: 16.9s	remaining: 32.9s
340:	learn: 0.0656260	total: 17s	remaining: 32.8s
341:	learn: 0.0655810	total: 17s	remaining: 32.7s
342:	learn: 0.065533

484:	learn: 0.0600508	total: 23.6s	remaining: 25.1s
485:	learn: 0.0599851	total: 23.7s	remaining: 25s
486:	learn: 0.0599476	total: 23.7s	remaining: 25s
487:	learn: 0.0599215	total: 23.7s	remaining: 24.9s
488:	learn: 0.0599022	total: 23.8s	remaining: 24.8s
489:	learn: 0.0598901	total: 23.8s	remaining: 24.8s
490:	learn: 0.0598789	total: 23.9s	remaining: 24.7s
491:	learn: 0.0598260	total: 23.9s	remaining: 24.7s
492:	learn: 0.0597992	total: 23.9s	remaining: 24.6s
493:	learn: 0.0597657	total: 24s	remaining: 24.6s
494:	learn: 0.0597491	total: 24s	remaining: 24.5s
495:	learn: 0.0597282	total: 24.1s	remaining: 24.5s
496:	learn: 0.0596853	total: 24.1s	remaining: 24.4s
497:	learn: 0.0596790	total: 24.2s	remaining: 24.3s
498:	learn: 0.0596411	total: 24.2s	remaining: 24.3s
499:	learn: 0.0596105	total: 24.3s	remaining: 24.3s
500:	learn: 0.0595704	total: 24.3s	remaining: 24.2s
501:	learn: 0.0595283	total: 24.4s	remaining: 24.2s
502:	learn: 0.0594630	total: 24.4s	remaining: 24.1s
503:	learn: 0.059445

643:	learn: 0.0550381	total: 31.2s	remaining: 17.3s
644:	learn: 0.0550286	total: 31.3s	remaining: 17.2s
645:	learn: 0.0549655	total: 31.3s	remaining: 17.2s
646:	learn: 0.0549531	total: 31.4s	remaining: 17.1s
647:	learn: 0.0549364	total: 31.4s	remaining: 17.1s
648:	learn: 0.0548924	total: 31.5s	remaining: 17s
649:	learn: 0.0548639	total: 31.5s	remaining: 17s
650:	learn: 0.0548163	total: 31.6s	remaining: 16.9s
651:	learn: 0.0547914	total: 31.6s	remaining: 16.9s
652:	learn: 0.0547702	total: 31.7s	remaining: 16.8s
653:	learn: 0.0547382	total: 31.7s	remaining: 16.8s
654:	learn: 0.0547180	total: 31.8s	remaining: 16.7s
655:	learn: 0.0547050	total: 31.8s	remaining: 16.7s
656:	learn: 0.0546842	total: 31.9s	remaining: 16.6s
657:	learn: 0.0546630	total: 31.9s	remaining: 16.6s
658:	learn: 0.0546221	total: 32s	remaining: 16.5s
659:	learn: 0.0546095	total: 32s	remaining: 16.5s
660:	learn: 0.0545987	total: 32.1s	remaining: 16.4s
661:	learn: 0.0545847	total: 32.1s	remaining: 16.4s
662:	learn: 0.054550

805:	learn: 0.0505714	total: 38.9s	remaining: 9.36s
806:	learn: 0.0505610	total: 38.9s	remaining: 9.31s
807:	learn: 0.0505353	total: 39s	remaining: 9.26s
808:	learn: 0.0505076	total: 39s	remaining: 9.21s
809:	learn: 0.0504591	total: 39.1s	remaining: 9.17s
810:	learn: 0.0504468	total: 39.1s	remaining: 9.12s
811:	learn: 0.0504244	total: 39.2s	remaining: 9.07s
812:	learn: 0.0504102	total: 39.2s	remaining: 9.02s
813:	learn: 0.0503863	total: 39.3s	remaining: 8.97s
814:	learn: 0.0503793	total: 39.3s	remaining: 8.92s
815:	learn: 0.0503502	total: 39.4s	remaining: 8.88s
816:	learn: 0.0503289	total: 39.4s	remaining: 8.83s
817:	learn: 0.0503216	total: 39.5s	remaining: 8.78s
818:	learn: 0.0502959	total: 39.5s	remaining: 8.73s
819:	learn: 0.0502383	total: 39.5s	remaining: 8.68s
820:	learn: 0.0502129	total: 39.6s	remaining: 8.63s
821:	learn: 0.0501999	total: 39.6s	remaining: 8.58s
822:	learn: 0.0501829	total: 39.7s	remaining: 8.54s
823:	learn: 0.0501639	total: 39.7s	remaining: 8.49s
824:	learn: 0.05

967:	learn: 0.0469388	total: 46.5s	remaining: 1.54s
968:	learn: 0.0469106	total: 46.6s	remaining: 1.49s
969:	learn: 0.0468986	total: 46.6s	remaining: 1.44s
970:	learn: 0.0468733	total: 46.7s	remaining: 1.39s
971:	learn: 0.0468603	total: 46.7s	remaining: 1.34s
972:	learn: 0.0468436	total: 46.8s	remaining: 1.3s
973:	learn: 0.0468137	total: 46.8s	remaining: 1.25s
974:	learn: 0.0468002	total: 46.8s	remaining: 1.2s
975:	learn: 0.0467940	total: 46.9s	remaining: 1.15s
976:	learn: 0.0467708	total: 46.9s	remaining: 1.1s
977:	learn: 0.0467539	total: 47s	remaining: 1.06s
978:	learn: 0.0467254	total: 47.1s	remaining: 1.01s
979:	learn: 0.0466988	total: 47.2s	remaining: 962ms
980:	learn: 0.0466813	total: 47.2s	remaining: 914ms
981:	learn: 0.0466632	total: 47.3s	remaining: 866ms
982:	learn: 0.0466476	total: 47.3s	remaining: 818ms
983:	learn: 0.0466237	total: 47.3s	remaining: 770ms
984:	learn: 0.0466104	total: 47.4s	remaining: 722ms
985:	learn: 0.0465985	total: 47.4s	remaining: 674ms
986:	learn: 0.046

## valid 예측

In [31]:
# pred_cat = cat.predict(x_valid)
# confusion_matrix(y_valid, pred_cat)

In [32]:
# print("Accuracy: ",accuracy_score(y_valid, pred_cat))
# print("Precision: ",precision_score(y_valid, pred_cat))
# print("Recall: ", recall_score(y_valid, pred_cat))
# print("F1 Score: ", f1_score(y_valid, pred_cat))
# print("ROC-AUC Score: ",roc_auc_score(y_valid, pred_cat))

## test 예측

In [73]:
pred_cat = cat.predict(x_test)
confusion_matrix(y_test, pred_cat)

array([[56834,   448],
       [ 1170,   102]], dtype=int64)

In [74]:
print("Accuracy: ",accuracy_score(y_test, pred_cat))
print("Precision: ",precision_score(y_test, pred_cat))
print("Recall: ", recall_score(y_test, pred_cat))
print("F1 Score: ", f1_score(y_test, pred_cat))
print("ROC-AUC Score: ",roc_auc_score(y_test, pred_cat))

Accuracy:  0.9723673873689244
Precision:  0.18545454545454546
Recall:  0.08018867924528301
F1 Score:  0.1119648737650933
ROC-AUC Score:  0.5361838616365378


In [75]:
print(classification_report(y_test, pred_cat))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     57282
           1       0.19      0.08      0.11      1272

    accuracy                           0.97     58554
   macro avg       0.58      0.54      0.55     58554
weighted avg       0.96      0.97      0.97     58554



## feature importance 뽑기

In [76]:
importances = cat.feature_importances_

In [77]:
ddf.iloc[:,:]

,id_seq,default_tf,ceo_name_cnt,bsn_opr_month,count,inno_cert_date_cnt,venture_cert_date_cnt,exporter_tf,importer_tf,research_lab_seq_tf,...,"research_lab_type_전담부서, 출판","research_lab_type_전담부서, 출판, 영상, 방송통신 및 정보서비스","research_lab_type_전담부서, 화학","research_lab_type_전담부서, 환경","research_lab_type_전문, 과학 및 기술 서비스",research_lab_type_정보처리,"research_lab_type_출판, 영상, 방송통신 및 정보서비스","research_lab_type_하수·폐기물처리, 원료재생 및 환경복원",research_lab_type_화학,research_lab_type_환경
1,769252,0,1,6.522093,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,769252,0,1,6.522093,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,769252,0,1,6.522093,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,769252,0,1,6.522093,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,769252,0,1,6.522093,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238666,969194,0,1,4.174387,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
238667,969194,0,1,4.174387,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
238668,969194,0,1,4.174387,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
238669,969194,0,1,4.174387,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
len(importances)

1534

In [79]:
lst = ddf.iloc[:,2:].dot(importances)
ddf['dot_sum']= lst

In [80]:
train_df.head(2)

,default_tf,ceo_name_cnt,bsn_opr_month,count,inno_cert_date_cnt,venture_cert_date_cnt,exporter_tf,importer_tf,research_lab_seq_tf,lab_cnt,...,"research_lab_type_전담부서, 출판","research_lab_type_전담부서, 출판, 영상, 방송통신 및 정보서비스","research_lab_type_전담부서, 화학","research_lab_type_전담부서, 환경","research_lab_type_전문, 과학 및 기술 서비스",research_lab_type_정보처리,"research_lab_type_출판, 영상, 방송통신 및 정보서비스","research_lab_type_하수·폐기물처리, 원료재생 및 환경복원",research_lab_type_화학,research_lab_type_환경
1,0,1,6.522093,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,6.522093,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
test_df.head(2)

,default_tf,ceo_name_cnt,bsn_opr_month,count,inno_cert_date_cnt,venture_cert_date_cnt,exporter_tf,importer_tf,research_lab_seq_tf,lab_cnt,...,"research_lab_type_전담부서, 출판","research_lab_type_전담부서, 출판, 영상, 방송통신 및 정보서비스","research_lab_type_전담부서, 화학","research_lab_type_전담부서, 환경","research_lab_type_전문, 과학 및 기술 서비스",research_lab_type_정보처리,"research_lab_type_출판, 영상, 방송통신 및 정보서비스","research_lab_type_하수·폐기물처리, 원료재생 및 환경복원",research_lab_type_화학,research_lab_type_환경
179851,0,1,3.931826,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179852,0,1,3.931826,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
ddf.to_csv('./cat_new_track3-3.csv')

In [83]:
importances

array([ 1.25352708, 13.88621124,  0.08888817, ...,  0.        ,
        0.        ,  0.        ])

In [84]:
indices = np.argsort(importances)[::-1]

print("Feature Ranking :")
importan = []
for f in range(x_train.shape[1]):
  print("{}. {} - {:.6f}".format(f + 1, x_train.columns[indices][f], importances[indices[f]]))
  importan.append(importances[indices[f]])

Feature Ranking :
1. bsn_opr_month - 13.886211
2. pulse_grade - 9.856492
3. bsn_history_cnt - 6.868302
4. pblntf_pclnd - 4.497785
5. total - 4.209035
6. avg - 4.203208
7. lab_cnt - 3.488759
8. research_lab_type_0 - 2.836721
9. research_lab_seq_tf - 2.375296
10. ass_retire_ratio - 2.245802
11. finance_seq_tf - 1.834318
12. nps_end_opr_month - 1.445934
13. emp_inde_crease_cnt - 1.379839
14. ceo_name_cnt - 1.253527
15. manpower_seq_tf - 0.812704
16. address_경기도 하남시 - 0.732270
17. importer_tf - 0.577423
18. address_경기도 화성시 - 0.569529
19. exporter_tf - 0.545211
20. article_cnt - 0.508067
21. ksic_code_0 - 0.507617
22. ksic_code_G46329 - 0.500866
23. address_강원도 강릉시 - 0.486105
24. address_경기도 수원시 - 0.482488
25. design_cnt - 0.472544
26. address_서울특별시 영등포구 - 0.467821
27. address_서울특별시 성동구 - 0.460493
28. stream_month_tf - 0.450952
29. ksic_code_G47912 - 0.449174
30. address_경기도 안양시 - 0.445313
31. growth_up_tf - 0.408402
32. address_서울특별시 금천구 - 0.401207
33. address_서울특별시 서대문구 - 0.397374
34. res

1354. ksic_code_G46732 - 0.000000
1355. ksic_code_G46733 - 0.000000
1356. ksic_code_G46739 - 0.000000
1357. ksic_code_G46742 - 0.000000
1358. ksic_code_G46750 - 0.000000
1359. ksic_code_G46799 - 0.000000
1360. ksic_code_G47 - 0.000000
1361. ksic_code_G47122 - 0.000000
1362. ksic_code_G47129 - 0.000000
1363. ksic_code_G46622 - 0.000000
1364. ksic_code_G46611 - 0.000000
1365. ksic_code_G46411 - 0.000000
1366. ksic_code_G46596 - 0.000000
1367. ksic_code_G46412 - 0.000000
1368. ksic_code_G46415 - 0.000000
1369. ksic_code_G46419 - 0.000000
1370. ksic_code_G46442 - 0.000000
1371. ksic_code_G46444 - 0.000000
1372. ksic_code_G46452 - 0.000000
1373. ksic_code_G46462 - 0.000000
1374. ksic_code_G46492 - 0.000000
1375. ksic_code_G46532 - 0.000000
1376. ksic_code_G46533 - 0.000000
1377. ksic_code_G46539 - 0.000000
1378. ksic_code_G46591 - 0.000000
1379. ksic_code_G46592 - 0.000000
1380. ksic_code_G46593 - 0.000000
1381. ksic_code_G46594 - 0.000000
1382. ksic_code_H52919 - 0.000000
1383. ksic_code_H